In [127]:
import nltk
from nltk import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajarshidas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [130]:
# file='Harry_Potter_Text/'+'Book1'+'.txt'
# with open(file) as f:
#     Lines = f.readlines()
#     book=""
#     for lin in Lines:
#         if not (len(lin)>5 and lin[:6]=='Page |'):
#             book+=lin.lower()
#     x=sent_tokenize(book)
#     y=[]
#     for l in x:
#         li=''.join(l.splitlines())
#         if not li[-1].isalnum():
#             li=li[:-1]
#         li= '<s> '+li+' <e>'
#         y.append(li)
#     w=word_tokenize(book)

In [206]:
text=""
for i in range (1,7):
    filename='Harry_Potter_Text/'+'Book'+str(i)+'.txt'
    with open(filename) as f:
        Lines = f.readlines()
        # book=""
        for lin in Lines:
            if not (len(lin)>5 and lin[:6]=='Page |'):
                if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                    text+=lin.lower()
        
    #     book = f.read()
    #     text = text + book.lower()
x=sent_tokenize(text)
sentences=[]
sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    sentences.append(li)
    sentencewise_tokens.append(word_tokenize(li))
tokens=word_tokenize(text)



In [207]:
len(sentences)

51670

In [208]:
len(tokens)

1155432

In [209]:
len(sentencewise_tokens)

51670

In [210]:
final_tokens=[]
for token in tokens:
    if not (len(token)==1 and not token.isalnum()):
        final_tokens.append(token)
tokens=final_tokens
#REMOVE PUNCTUATION

In [211]:
tokens

['the',
 'boy',
 'who',
 'lived',
 'mr.',
 'and',
 'mrs.',
 'dursley',
 'of',
 'number',
 'four',
 'privet',
 'drive',
 'were',
 'proud',
 'to',
 'say',
 'that',
 'they',
 'were',
 'perfectly',
 'normal',
 'thank',
 'you',
 'very',
 'much',
 'they',
 'were',
 'the',
 'last',
 'people',
 'you',
 'd',
 'expect',
 'to',
 'be',
 'involved',
 'in',
 'anything',
 'strange',
 'or',
 'mysterious',
 'because',
 'they',
 'just',
 'didn',
 't',
 'hold',
 'with',
 'such',
 'nonsense',
 'mr.',
 'dursley',
 'was',
 'the',
 'director',
 'of',
 'a',
 'firm',
 'called',
 'grunnings',
 'which',
 'made',
 'drills',
 'he',
 'was',
 'a',
 'big',
 'beefy',
 'man',
 'with',
 'hardly',
 'any',
 'neck',
 'although',
 'he',
 'did',
 'have',
 'a',
 'very',
 'large',
 'mustache',
 'mrs.',
 'dursley',
 'was',
 'thin',
 'and',
 'blonde',
 'and',
 'had',
 'nearly',
 'twice',
 'the',
 'usual',
 'amount',
 'of',
 'neck',
 'which',
 'came',
 'in',
 'very',
 'useful',
 'as',
 'she',
 'spent',
 'so',
 'much',
 'of',
 'he

In [212]:
vocab=list(set(tokens))

In [213]:
print(len(vocab))

22717


In [214]:
n_limit=5

In [215]:
# ngrams=[]
# for i in range (n_limit):
#     d={}
#     ngrams.append(d)
# for n in range (1,n_limit+1):
#     for i in range(len(tokens)):
#         if i+n>len(tokens):
#             break
#         lis=[]
#         for j in range(i,i+n):
#             lis.append(tokens[j])
#         lis=tuple(lis)
#         if lis not in ngrams[n-1]:
#             ngrams[n-1][lis] = 1
#         else : 
#             ngrams[n-1][lis] += 1
#     ngrams[n-1]=dict(sorted(ngrams[n-1].items(), key=lambda item: -item[1]))
#     print(n,len(ngrams[n-1]))




In [216]:
ngrams=[]
for i in range (n_limit):
    d={}
    ngrams.append(d)
for n in range (1,n_limit+1):
    for i in range(len(tokens)):
        if i+n>len(tokens):
            break
        lis=[]
        for j in range(i,i+n):
            lis.append(tokens[j])
        lis=tuple(lis)
        if lis not in ngrams[n-1]:
            ngrams[n-1][lis] = 1
        else : 
            ngrams[n-1][lis] += 1
    ngrams[n-1]=dict(sorted(ngrams[n-1].items(), key=lambda item: -item[1]))
    print(n,len(ngrams[n-1]))




1 22717
2 282477
3 645139
4 836614
5 896738


In [222]:
ngrams[1]

{('of', 'the'): 3901,
 ('in', 'the'): 2915,
 ('said', 'harry'): 2265,
 ('he', 'was'): 2063,
 ('at', 'the'): 2030,
 ('on', 'the'): 1933,
 ('to', 'the'): 1900,
 ('it', 'was'): 1716,
 ('he', 'had'): 1705,
 ('out', 'of'): 1581,
 ('don', 't'): 1523,
 ('it', 's'): 1379,
 ('said', 'ron'): 1323,
 ('to', 'be'): 1255,
 ('into', 'the'): 1215,
 ('didn', 't'): 1212,
 ('harry', 's'): 1209,
 ('in', 'a'): 1159,
 ('he', 'said'): 1122,
 ('i', 'm'): 1075,
 ('said', 'hermione'): 1059,
 ('had', 'been'): 1005,
 ('from', 'the'): 1003,
 ('was', 'a'): 998,
 ('of', 'his'): 973,
 ('he', 's'): 962,
 ('as', 'though'): 953,
 ('and', 'the'): 929,
 ('there', 'was'): 927,
 ('as', 'he'): 918,
 ('with', 'a'): 896,
 ('in', 'his'): 864,
 ('going', 'to'): 825,
 ('that', 'he'): 806,
 ('harry', 'and'): 774,
 ('and', 'hermione'): 759,
 ('for', 'the'): 749,
 ('for', 'a'): 744,
 ('they', 'were'): 717,
 ('you', 're'): 713,
 ('and', 'he'): 705,
 ('he', 'could'): 702,
 ('of', 'them'): 696,
 ('ron', 'and'): 687,
 ('i', 've'): 680,


# SENTENCE GENERATION

In [257]:
def get_n(n,a,line,word_limit):
    if word_limit==0:
        print(line)
        return
    sz=len(a)
    if sz==0 or n==1:
        word=next(iter(ngrams[0]))[0]#first key
        a.append(word)
        line=line+' '+word
        # print(line)
        get_n(n,a,line,word_limit-1)
    elif sz<n-1:
        found = False
        for key in ngrams[sz]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                # a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
    elif sz==n-1:
        found = False
        for key in ngrams[n-1]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
        




In [261]:
def generate_sentence(n,word_limit):
    prev_tokens=[]
    sentence=""
    get_n(n,prev_tokens,sentence,word_limit)     


In [268]:
generate_sentence(5,100)

 the door and turned the key fumbling in their panic harry pulled the door open and they ran inside hermione granger was shrinking against the wall opposite looking as if she was about to faint the troll was advancing on her knocking the sinks off the walls as it went confuse it harry said desperately to ron and seizing a tap he threw it as hard as he could into the kitchen harry hurried into the living room in time to catch the last report on the evening news and finally bird-watchers everywhere have reported that the nation s owls


# PERPLEXITY